In [14]:
import serial
import pandas as pd
from datetime import datetime, timedelta
import os

# Configure the serial port and baud rate
ser = serial.Serial('/dev/ttyUSB0', 115200, timeout=1)

# Load or create the attendance DataFrame
try:
    df = pd.read_excel('attendance.xlsx', index_col=0)
except FileNotFoundError:
    df = pd.DataFrame(columns=['QR Code', 'Name'])

# Define a function to determine the current week number
def get_week_number(date):
    start_date = datetime(2024, 7, 17)  # Change to the start date of your term
    return ((date - start_date).days // 7) + 1

# Specify the path for the Excel file
output_path = os.path.join(os.getcwd(), 'attendance.xlsx')
print(f"Excel file will be saved to: {output_path}")

try:
    while True:
        line = ser.readline().decode('utf-8').strip()
        if line and line.startswith("Found '") and line.endswith("'"):
            qr_code = line[7:-1]  # Extract the QR code content
            print(f"Received QR code: {qr_code}")  # Debugging statement
            timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            date_now = datetime.now()
            week_number = get_week_number(date_now)
            week_col = f'Week {week_number}'

            if qr_code not in df['QR Code'].values:
                print(f"QR code {qr_code} not found. Asking for name.")  # Debugging statement
                name = input(f"Enter name for QR code {qr_code}: ")  # For the first-time entry
                new_entry = pd.DataFrame({'QR Code': [qr_code], 'Name': [name]})
                df = pd.concat([df, new_entry], ignore_index=True)
                print(f"Added new entry: {new_entry}")  # Debugging statement
            
            if week_col not in df.columns:
                df[week_col] = 'Absent'  # Default to 'Absent' for all initially
                print(f"Added new column for {week_col}")  # Debugging statement
            
            df.loc[df['QR Code'] == qr_code, week_col] = 'Present'
            print(f"Marked {qr_code} as present for {week_col}")  # Debugging statement
            
            print("Saving to Excel...")  # Debugging statement before saving
            df.to_excel(output_path)
            print("Excel file saved.")  # Debugging statement after saving
            print(f'Logged: {qr_code} at {timestamp} for {week_col}')
except KeyboardInterrupt:
    print("\nInterrupt received. Saving Excel file and exiting...")
    df.to_excel(output_path)
    print("Excel file saved.")


Excel file will be saved to: /home/studio-one/Documents/StudioRepo/ScannerProject/attendance.xlsx
Received QR code: Sn
QR code Sn not found. Asking for name.
Added new entry:   QR Code Name
0      Sn   sn
Marked Sn as present for Week 1
Saving to Excel...
Excel file saved.
Logged: Sn at 2024-07-19 21:50:40 for Week 1
Received QR code: Sn
Marked Sn as present for Week 1
Saving to Excel...
Excel file saved.
Logged: Sn at 2024-07-19 21:50:56 for Week 1
Received QR code: Sn
Marked Sn as present for Week 1
Saving to Excel...
Excel file saved.
Logged: Sn at 2024-07-19 21:50:56 for Week 1

Interrupt received. Saving Excel file and exiting...
Excel file saved.
